<a href="https://colab.research.google.com/github/kangelaa/pic16b-final-proj/blob/main/pipeline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/usr/local/etc/jupyter/jupyter_notebook_config.d/panel-client-jupyter.json
    	/usr/local/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/usr/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/root/.local/etc/jupyter/jupyter_notebook_config.json
|DEBUG|Paths used for configuration of jupyter_notebook_config: 
    	/root/.jupyter/jupyter_notebook_config.json
|INFO|google.colab serverextension initialized.
|INFO|Serving notebooks from local directory: /content
|INFO|Jupyter Notebook 6.4.8 is running at:
|INFO|http://localhost:8888/?token=80780f6077be101e2c238b2c2445d64d1da485bc4e2cdead
|INFO| or http://127.0.0.1:8888/?token=80780f6077be101e2c238b2c2445d64d1da485bc4e2cdead
|INFO|Use Con

In [ ]:
from bs4 import BeautifulSoup
import requests

def GetProxys():
    website_html = requests.get("https://free-proxy-list.net").text
    soup = BeautifulSoup(website_html, "html.parser")
    soup = soup.find("table")
    Headings = []
    Body = []
    Ip = []
    for heading in soup.find_all("th"):
        Headings.append(heading.text)
    for ip_html in soup.find_all("tr"):
        ip_info_list = []
        for ip_info in ip_html.find_all("td"):
            ip_info_list.append(ip_info.text)
        if len(ip_info_list) > 1:
            if ip_info_list[6] == "yes":
                Ip.append(f"{ip_info_list[0]}:{ip_info_list[1]}")
                Body.append(ip_info_list)
    return Ip, 


ips = GetProxys()
ips

([],)

In [ ]:
def perform_request(ip):
    http_proxy = f"http://{ip}"
    https_proxy = f"https://{ip}"

    proxyDict = {
        "http": http_proxy,
        "https": https_proxy,
    }
    try:
       r = requests.get("http://api.ipify.org",
                             proxies=proxyDict, timeout=10)
       print(f"{r}/{r.text}", end="/")
    except:
        print("{-_-}", end="/")
    print()

In [ ]:
for ip in ips:
    perform_request(ip)

{-_-}/


In [ ]:
#install required packages
!pip install textblob
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 6.4 MB/s eta 0:00:00


In [ ]:
#imports
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.oauth2 as oauth2
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re
import json
import csv
import time

## Part 1: Turn data from json to csv

In [ ]:
#access files from Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#just try for one .json file right now

# with open('/content/gdrive/MyDrive/2022-23/Final Project/mpd.slice.0-999.json', 'r') as json_file:
#     data = json.load(json_file)



playlists = data['playlists']

tracks = []

for playlist in playlists:
    playlist_name = playlist['name']
    playlist_tracks = playlist['tracks']
    
    for track in playlist_tracks:
     
        track_info = {
            'playlist_name': playlist_name,
            'track_name': track['track_name'],
            'artist_name': track['artist_name'],
            'album_name': track['album_name'],
            'track_uri': track['track_uri']
        }

        tracks.append(track_info)

fieldnames = ['playlist_name', 'track_name', 'artist_name', 'album_name', 'track_uri']

with open('output.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(tracks)

In [ ]:
rawdf = pd.read_csv('output.csv')

In [ ]:
rawdf.head(10)

,playlist_name,track_name,artist_name,album_name,track_uri
0,Throwbacks,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,Throwbacks,Toxic,Britney Spears,In The Zone,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,Throwbacks,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,Throwbacks,Rock Your Body,Justin Timberlake,Justified,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,Throwbacks,It Wasn't Me,Shaggy,Hot Shot,spotify:track:1lzr43nnXAijIGYnCT8M8H
5,Throwbacks,Yeah!,Usher,Confessions,spotify:track:0XUfyU2QviPAs6bxSpXYG4
6,Throwbacks,My Boo,Usher,Confessions,spotify:track:68vgtRHr7iZHpzGpon6Jlo
7,Throwbacks,Buttons,The Pussycat Dolls,PCD,spotify:track:3BxWKCI06eQ5Od8TY2JBeA
8,Throwbacks,Say My Name,Destiny's Child,The Writing's On The Wall,spotify:track:7H6ev70Weq6DdpZyyTmUXk
9,Throwbacks,Hey Ya! - Radio Mix / Club Mix,OutKast,Speakerboxxx/The Love Below,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM


In [ ]:
rawdf.shape

(67503, 5)

## Part 2: Data Preprocessing and Additional Data Acquisition

### Remove duplicates

In [ ]:
# Drop song duplicates
def drop_duplicates(df):
    '''
    Input: full dataframe of songs with possible duplicates
    Output: dataframe of songs with duplicates dropped
    '''
    df['songartistconcat'] = df.apply(lambda row: row['artist_name']+row['track_name'],axis = 1)
    return df.drop_duplicates('songartistconcat')

rawdf= drop_duplicates(rawdf)

#test
print("Are all songs unique: ",len(pd.unique(rawdf.songartistconcat))==len(rawdf))

Are all songs unique:  True


In [ ]:
rawdf.shape
#rawdf.head()


(34250, 6)

In [ ]:
# get audio features from track uri and create a data frame with artist/track name plus audio features

client_id = '4a8cd8bc834b4c2bb0aac4cc89d4d990'
client_secret = '321d379d39964c94810f3bb52e8915b1'
redirect_uri = 'http://localhost:3001'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

track_uris = rawdf['track_uri'].tolist()

In [ ]:
#get 100 track_ids for audio_features() function
hundred_uri_chunks = [track_uris[i:i + 100] for i in range(0, len(track_uris), 100)]

audio_features = []

for chunk in hundred_uri_chunks:
    chunk_features = sp.audio_features(chunk)
    audio_features.extend(chunk_features)

df_audio_features = pd.concat([rawdf, pd.DataFrame(audio_features)], axis=1)

In [ ]:
#get 50 track_ids at a time for tracks(),  
fifty_uri_chunks = [track_uris[i:i + 50] for i in range(0, len(track_uris), 50)]

misc_info = []

for i, chunk in enumerate(fifty_uri_chunks):
    chunk_tracks = sp.tracks(chunk)['tracks']
    misc_info.extend(chunk_tracks)
    print(chunk_tracks)
    time.sleep(5)
    

#print(misc_info.tail())
misc_info=pd.DataFrame(list(map(dict,misc_info)))
#from operator import itemgetter
track_popularity=list(map(itemgetter(N),misc_info))
track_popularity=misc_info["popularity"]
artists=misc_info["artists"]
artist_genres = []
artist_popularity = []
artist_ids = []
for artist in artists:
  #take main artist for each song
  artist_ids.append(artist[0]["id"])

artist_info=sp.artists(artist_ids)
artist_info=pd.DataFrame(artist_info["artists"])
artist_popularity=artist_info["popularity"]
artist_genres=artist_info["genres"]

KeyboardInterrupt: ignored

In [ ]:
#concatenate all the relevant columns together 
df_full = pd.concat([df_audio_features, artist_genres, artist_popularity], axis=1)

In [ ]:
df_full.shape

(47544, 26)

In [ ]:
df_full.head()

,playlist_name,track_name,artist_name,album_name,track_uri,songartistconcat,danceability,energy,key,loudness,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genres,popularity
0,Throwbacks,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,Missy ElliottLose Control (feat. Ciara & Fat M...,0.904,0.813,4.0,-7.105,...,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72.0
1,Throwbacks,Toxic,Britney Spears,In The Zone,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,Britney SpearsToxic,0.774,0.838,5.0,-3.914,...,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0,"[dance pop, pop]",80.0
2,Throwbacks,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),spotify:track:0WqIKmW4BTrj3eJFmnCKMv,BeyoncéCrazy In Love,0.664,0.758,2.0,-6.583,...,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933.0,4.0,"[pop, r&b]",87.0
3,Throwbacks,Rock Your Body,Justin Timberlake,Justified,spotify:track:1AWQoqb9bSvzTjaLralEkT,Justin TimberlakeRock Your Body,0.892,0.714,4.0,-6.055,...,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267.0,4.0,"[dance pop, pop]",80.0
4,Throwbacks,It Wasn't Me,Shaggy,Hot Shot,spotify:track:1lzr43nnXAijIGYnCT8M8H,ShaggyIt Wasn't Me,0.853,0.606,0.0,-4.596,...,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600.0,4.0,"[dance pop, pop rap, reggae fusion]",73.0


### Feature Selection

In [ ]:
df_full = drop_duplicates(df_full)
print("Are all songs unique: ",len(pd.unique(df_full.songartistconcat))==len(df_full))

drop=['track_uri','songartistconcat','type','uri','track_href','analysis_url','duration_ms',]
df_full=df_full.drop(drop,axis=1)


Are all songs unique:  True


In [ ]:
df_full.shape

(34251, 19)

In [ ]:
#drop na's, just in case
df=df_full.dropna()
df.shape

(40, 26)

In [ ]:
#convert genres column from string to list of genres

df['genres_list']=df['genres'].apply(lambda x: str(x).split(", "))

print(len(df['genres_list'][0]))

9


0        [['dance pop', 'hip hop', 'hip pop', 'neo soul...
1                                   [['dance pop', 'pop']]
2                                         [['pop', 'r&b']]
3                                   [['dance pop', 'pop']]
4              [['dance pop', 'pop rap', 'reggae fusion']]
                               ...                        
67499                                                [nan]
67500                                                [nan]
67501                                                [nan]
67502                                                [nan]
51                                                   [nan]
Name: genres_list, Length: 34251, dtype: object

## Part 3: Feature Engineering

In [ ]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create one-hot-encoded features of a specific column
    
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    #get_dummies() converts categorical variable into dummy/indicator variables
    tf_df = pd.get_dummies(df[column])
    print(tf_df)
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [ ]:
type(str(df['genres'][0]))

str

Genres - one hot encoding, tf-idf version

In [ ]:
# TF-IDF implementation: find most important genre for each song and that genre's prevalence across all songs to weight genre accordingly

#function from scikit-learn
tfidf = TfidfVectorizer()

#get weighted values for each genre
tfidf_matrix =  tfidf.fit_transform(df['genres'].apply(lambda x: " ".join(str(x))))
genre_df = pd.DataFrame(tfidf_matrix.toarray())

genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]
if 'genre|unknown' in genre_df.columns:
  genre_df.drop(columns='genre|unknown') # Drop unknown genre

#delete index col
genre_df.reset_index(drop = True, inplace=True)

#view for first row
print(genre_df.iloc[0])

ValueError: ignored

In [ ]:
df.head(100)

,playlist_name,track_name,artist_name,album_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,time_signature,genres,popularity,genres_list
0,Throwbacks,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,The Cookbook,0.904,0.813,4.0,-7.105,0.0,0.1210,0.03110,0.006970,0.0471,0.810,125.461,0UaMYEvWZi0ZqiDOoHU3YI,4.0,"[dance pop, hip hop, hip pop, neo soul, pop ra...",72.0,"[['dance pop', 'hip hop', 'hip pop', 'neo soul..."
1,Throwbacks,Toxic,Britney Spears,In The Zone,0.774,0.838,5.0,-3.914,0.0,0.1140,0.02490,0.025000,0.2420,0.924,143.040,6I9VzXrHxO9rA9A5euc8Ak,4.0,"[dance pop, pop]",80.0,"[['dance pop', 'pop']]"
2,Throwbacks,Crazy In Love,Beyoncé,Dangerously In Love (Alben für die Ewigkeit),0.664,0.758,2.0,-6.583,0.0,0.2100,0.00238,0.000000,0.0598,0.701,99.259,0WqIKmW4BTrj3eJFmnCKMv,4.0,"[pop, r&b]",87.0,"[['pop', 'r&b']]"
3,Throwbacks,Rock Your Body,Justin Timberlake,Justified,0.892,0.714,4.0,-6.055,0.0,0.1410,0.20100,0.000234,0.0521,0.817,100.972,1AWQoqb9bSvzTjaLralEkT,4.0,"[dance pop, pop]",80.0,"[['dance pop', 'pop']]"
4,Throwbacks,It Wasn't Me,Shaggy,Hot Shot,0.853,0.606,0.0,-4.596,1.0,0.0713,0.05610,0.000000,0.3130,0.654,94.759,1lzr43nnXAijIGYnCT8M8H,4.0,"[dance pop, pop rap, reggae fusion]",73.0,"[['dance pop', 'pop rap', 'reggae fusion']]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,korean,Ah-Choo,Lovelyz,Lovelyz8,0.609,0.246,8.0,-8.449,1.0,0.0376,0.78400,0.000000,0.0825,0.371,123.773,06L1apH8kLF47dbhZ4Zg9A,4.0,NaN,NaN,[nan]
97,korean,BREATHE,LEE HI,SEOULITE,0.763,0.658,5.0,-5.643,0.0,0.1470,0.02580,0.000000,0.0915,0.389,80.038,2qWgqPdW1OiAP8KSBH1b93,4.0,NaN,NaN,[nan]
98,korean,FXXK WIT US,LEE HI,SEOULITE,0.604,0.875,7.0,-1.995,0.0,0.0539,0.24400,0.000000,0.1140,0.219,129.925,2PTf3zh9UUsgdxQ5b0eXg8,4.0,NaN,NaN,[nan]
99,korean,I Will Show You,Ailee,U&I,0.756,0.426,1.0,-8.750,0.0,0.0745,0.50400,0.039400,0.0884,0.355,135.960,7tIJDktakabGoHjwTTa35W,4.0,NaN,NaN,[nan]


Song and Artist Popularity -  normalization

In [ ]:
#artist normalization
artist_pop = df[["artist_popularity"]].reset_index(drop = True)
#from scikit-learn
scaler = MinMaxScaler()
artist_pop_scaled = pd.DataFrame(scaler.fit_transform(artist_pop), columns = artist_pop.columns)

print(artist_pop_scaled.head())

#track normalization
track_pop = df[["track_popularity"]].reset_index(drop = True)
#from scikit-learn
scaler = MinMaxScaler()
track_pop_scaled = pd.DataFrame(scaler.fit_transform(track_pop), columns = track_pop.columns)

print(track_pop_scaled.head())

KeyError: ignored

Audio features - normalization to largest and smallest values of each feature